In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tensorflow.keras.preprocessing import image
import glob
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, Input, Dense, Flatten, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import resnet50, vgg16

In [2]:
train_data_path = "../input/fer2013/train" 
test_data_path =  "../input/fer2013/test"

### Creating Generators

In [8]:
def create_generator(batch_size, train_data_path, test_data_path):
    
    target_size=(48,48)
    
    train_preprocesser = ImageDataGenerator(
        rescale = 1/255.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        validation_split=0.2
    )
    
    train_generator = train_preprocesser.flow_from_directory(
        train_data_path,
        class_mode = "categorical",
        target_size = target_size,
        shuffle = True,
        batch_size = batch_size,
        subset = "training"
    )
    
    val_generator = train_preprocesser.flow_from_directory(
        train_data_path,
        class_mode = "categorical",
        target_size = target_size,
        shuffle = True,
        batch_size = batch_size,
        subset = "validation"
    )
    
    test_preprocesser = ImageDataGenerator(rescale=1./255,
                                           width_shift_range=0.1,
                                           height_shift_range=0.1,
                                           horizontal_flip=True)
    
    test_generator = test_preprocesser.flow_from_directory(
        test_data_path,
        class_mode = "categorical",
        target_size=target_size,
        shuffle = True,
        batch_size = batch_size
    )
    
    return train_generator, val_generator, test_generator

In [9]:
batch_size = 32

train_generator, val_generator, test_generator = create_generator(batch_size, train_data_path, test_data_path)

In [10]:
train_generator.class_indices

### ResNet Model

In [16]:
def ResNet_model(number_of_classes):
    
    model = models.Sequential()
    
    pretrained_model = resnet50.ResNet50(include_top=False, 
                                         weights='imagenet', 
                                         classes = number_of_classes, 
                                         input_shape=(48,48,3))
    
    pretrained_model.trainable = True
    
    model.add(pretrained_model)
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(BatchNormalization())

    model.add(Dense(256,activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01)))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(7, activation="softmax"))
    
    return model

In [17]:
number_of_classes = train_generator.num_classes
model_ResNet50 = ResNet_model(number_of_classes)
model_ResNet50.summary()

In [18]:
epochs = 50

path_to_save_model_resnet = "resnet.h5"

ckpnt_saver = ModelCheckpoint(
    path_to_save_model_resnet,
    monitor = "val_accuracy",
    mode="max",
    save_best_only = True,
    save_freq = "epoch",
    verbose=1
    )

from tensorflow.keras.optimizers import Adam

model_ResNet50.compile(optimizer= Adam(1e-5),
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])

history_ResNet = model_ResNet50.fit(train_generator, 
                 epochs=epochs,
                 batch_size=32,
                 validation_data = val_generator,
                 callbacks = [ckpnt_saver]
                 )

In [21]:
#model_resnet = tf.keras.models.load_model("resnet.h5")

print("Evaluating train set: ")
model_ResNet50.evaluate(train_generator)

print("Evaluating test set: ")
model_ResNet50.evaluate(test_generator)

In [22]:
print(history_ResNet.history.keys())

In [23]:
def show_acc_and_loss(model):    
    plt.figure(figsize=(14,6))

    train_acc = model.history["accuracy"]
    train_loss = model.history["loss"]

    plt.subplot(1,2,1)
    plt.plot(model.history["accuracy"])
    plt.plot(model.history["val_accuracy"])
    plt.title("Train and Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(["Train", "Validation"])

    plt.subplot(1,2,2)
    plt.plot(model.history["loss"])
    plt.plot(model.history["val_loss"])
    plt.title("Train and Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(["Train", "Validation"])

    plt.show()

In [24]:
show_acc_and_loss(history_ResNet)

In [34]:
from sklearn.metrics import confusion_matrix


y_pred = model_ResNet50.predict(test_generator)
pred = np.argmax(y_pred, axis=1)

cm = confusion_matrix(test_generator.classes, pred)
plt.figure(figsize = (14,8))
plt.title("Confusion Matrix")

ax = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", annot_kws={"size": 12})